# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

SyntaxError: invalid syntax (1460578121.py, line 11)

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [3]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

NameError: name 'Dual' is not defined

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

## Тестирование

Для того, чтобы протестировать дуальные числа я решил воспользоваться их связью с диффериенцированием и проверить корректность арифметических операций для различных функций, охватывающих все добавленные операции. (см MyDualTests.py)

In [4]:
import unittest
import MyDualTests

unittest.TextTestRunner().run(unittest.makeSuite(MyDualTests.FirstTestCase))

........
----------------------------------------------------------------------
Ran 8 tests in 0.003s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [5]:
import unittest
import MyDualTests

unittest.TextTestRunner().run(unittest.makeSuite(MyDualTests.SecondTestCase))

....
----------------------------------------------------------------------
Ran 4 tests in 0.002s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [7]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

def f1(x: float) -> float:
    return math.sin(3*x) + 2*x**3 - (x*2 + 1)**0.5 + math.log(x + 1)
derivative(f, 2.)

/tmp/ipykernel_81973/2315536219.py:8: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  derivative(f, 2.)


22.0

In [8]:
import unittest
import MyDualTests

unittest.TextTestRunner().run(unittest.makeSuite(MyDualTests.NumericalDifferentiationTestCase))

....
----------------------------------------------------------------------
Ran 4 tests in 0.002s

OK


<unittest.runner.TextTestResult run=4 errors=0 failures=0>

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

 Генерацию случайных функций можно осуществить, например, двумя путями.
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [1]:
import unittest
import RandomFunctionsTests

unittest.TextTestRunner().run(unittest.makeSuite(RandomFunctionsTests.RandomGeneratorTestCase))

..........................F..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................E..........................................................................................................................................................

<unittest.runner.TextTestResult run=1178 errors=1 failures=1>

In [2]:
import RandomFunctionsGenerator

RandomFunctionsGenerator.generate_and_dump_to_files(20, "random_functions_dump_test.dat")

sin(x)+cos(sin(sin(log(-4*x))-8))-1-cos(x)/sin(-8*x**1-2*x**2-4)
cos(log(x))*-3*x+1*x*10*x*sin(x)-x/-6
-5*x-sin(-3*x**1+5*x**2-3)+9-x/-7
log(sin(x))-3*x*log(-4*x**1+9)+cos(-7*x**1-9)
cos(6*x**1+7*x**2+7)+log(sin(-9))+log(-9*x**1+1)+5*x*sin(10*x)
cos(x)/-1+cos(sin(-9*x))+x*5*8
3-6*x*-8*x-5/-8
log(-2*x**1-5)/x-7*x*x*-2
cos(cos(-3+4+2+7))-2*x*5*x/-5-5*x*x-1/-1
4-log(log(-3-10*x+sin(x)))*cos(sin(sin(-4*x)+5*x*x))+8*x*log(9*x**1-2)

The inequality, log(-10*x + sin(x) - 3) > 0, cannot be solved using
solve_univariate_inequality.
7*x*x-sin(7*x**1-10)-8*x*10*x*log(-2*x)
x/-4/2*x*cos(-8*x**1+2*x**2-1)*cos(-5*x**1-5*x**2+9)-5*x
cos(-4*x**1-9)*sin(5*x**1+7)-7*x*7-3
-2-3*x-2-10+6*x*7*x
-10*x/sin(sin(8-4*x*x+9+1))/cos(-9*x**1-4)/8-2*x*x-9
-7-cos(sin(9*x))*sin(1*x)-log(6*x)
log(8*x**1-3)*cos(x)-log(-7*x**1-10*x**2-1)-log(log(sin(10*x)-9+x))

The inequality, x + sin(10*x) - 9 > 0, cannot be solved using
solve_univariate_inequality.
x-5+log(cos(x))-9-4-log(cos(-2*x*6*x+7+x+5))

The inequality, cos(-12

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [3]:
from typing import Callable
from math import log
from MyDual import Dual

def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y

def f2(x: float, y: float, z: float, k: float) -> float:
    return x * y + z - 5 * y + k**2/10

def f0(x: float, z: float) -> float:
    return x**3 + 10**z/log(10)

def multiple_argument_diff(func: Callable[..., float]) -> Callable[..., float]:
    return lambda *args: [ func(*tuple(list(args[:index]) + [Dual(args[index],1)] + list(args[index+1:]))).d for index in range(len(args))]
print(multiple_argument_diff(f)(10, 10, 10))
print(multiple_argument_diff(f2)(10, 10, 10, 10))
print(multiple_argument_diff(f0)(2, 2))


[10.0, 5.0, 1]
[10.0, 5.0, 1, 2.0]
[12.0, 100.0]
